In [1]:
import numpy as np
import math
import random
import cv2

In [2]:
def calc_dist(city1, city2):
    return math.sqrt((city1.x - city2.x)**2 + (city1.y - city2.y)**2)

In [3]:
class City:
    def __init__(self, x=None, y=None):
        if x is None:
            self.x = int(random.uniform(0, width))
            self.y = int(random.uniform(0, height))
        else:
            self.x = x
            self.y = y

In [4]:
class Path:
    def __init__(self):
        self.order = [i for i in range(num_cities)]
        random.shuffle(self.order)
        self.fitness = None
        
    def calc_fitness(self):
        total_dist = 0
        for i in range(len(self.order) - 1):
            city1 = cities[self.order[i]]
            city2 = cities[self.order[i + 1]]
            dist = calc_dist(city1, city2)
            total_dist += dist
        if total_dist == 0:
            import pdb; pdb.set_trace()
        self.fitness = 1 / total_dist
        return self.fitness
    
    def mutate(self):
        ind1 = int(random.uniform(0, num_cities))
        ind2 = int(random.uniform(0, num_cities))
        # swap
        temp = self.order[ind1]
        self.order[ind1] = self.order[ind2]
        self.order[ind2]= temp
            

In [5]:
class Population:
    def __init__(self):
        self.paths = [Path() for i in range(pop_size)]
        self.best_path = self.paths[0]
        self.best_fit = self.paths[0].calc_fitness()
        
    def run(self):
        for path in self.paths:
            fitness = path.calc_fitness()
            if fitness > self.best_fit:
                self.best_fit = fitness
                self.best_path = path
                
    def evaluate(self):
        mating_pool = []
        for path in self.paths:
            num = int(math.pow(path.fitness / self.best_fit, 2) * 100)
            for i in range(num):
                mating_pool.append(path)
        self.mating_pool = mating_pool
        
    def selection(self):
        new_paths = []
        for i in range(pop_size):
            mate = self.mating_pool[int(random.uniform(0, len(self.mating_pool)))]
            val = random.uniform(0, 1)
            if val < mut_rate:
                mate.mutate()
            new_paths.append(mate)
        self.paths = new_paths
        
        # reset other params
        self.best_path = self.paths[0]
        self.best_fit = self.paths[0].calc_fitness()            
                

In [16]:
def get_cities(num_cities):
    cities = []
    cities_dict = {}
    count = 0
    while count != num_cities:
        x = int(random.uniform(city_thickness * 3, width - city_thickness * 3))
        y = int(random.uniform(city_thickness * 3, height - city_thickness * 3))
        if x not in cities_dict:
            cities.append(City(x, y))
            cities_dict[x] = { y: True }
            count += 1
        elif y not in cities_dict:
            cities.append(City(x, y))
            cities_dict[x][y] = True
            count += 1
    return cities

In [30]:
width = 500
height = 500
num_cities = 10
pop_size = 1000
mut_rate = 0.1
city_thickness = 3

In [31]:
generations = 0
cities = get_cities(num_cities)
population = Population()

total_best_fit = None
total_best_order = None

while True:
    population.run()
    population.evaluate()
    population.selection()
    
    # set up display
    canvas = np.zeros((height, width))
    for city in cities:
        canvas[city.y - city_thickness:city.y + city_thickness, city.x - city_thickness: city.x + city_thickness] = 1
    
    # visualize
    fit = population.best_fit
    path = population.best_path
    if total_best_fit is None or fit > total_best_fit:
        total_best_fit = fit
        total_best_order = path.order.copy()
        
    order = total_best_order
    for i in range(num_cities - 1):
        city1 = cities[order[i]]
        city2 = cities[order[i + 1]]
        # draw line
        cv2.line(canvas, (city1.x, city1.y), (city2.x, city2.y), (255, 0, 0), thickness=2)
        
        # draw order
        cv2.putText(canvas, str(i), (city1.x, city1.y - 3), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
    
    # draw order
    cv2.putText(canvas, str(num_cities - 1), (cities[order[-1]].x, cities[order[-1]].y - 3), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0), 1)
    
    # put generation
    cv2.putText(canvas, 'Generation: {}'.format(generations), (20, 20), cv2.FONT_HERSHEY_COMPLEX,0.5,(255,0,0),1)
    cv2.putText(canvas, 'Fitness: {}'.format(total_best_fit), (20, 45), cv2.FONT_HERSHEY_COMPLEX,0.5,(255,0,0),1)
    
    # display
    cv2.imshow('canvas', canvas)
    
    k = cv2.waitKey(30) & 0xFF
    if k == ord('q'):
        break
    
    generations += 1

cv2.destroyAllWindows()